In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision
from torchvision import datasets

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data", 
    train=False,
    download=True, 
    transform=torchvision.transforms.ToTensor()
)

100%|██████████| 26421880/26421880 [00:07<00:00, 3664321.23it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 197826.82it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3020379.51it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [19]:
batch_size = 8

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([8, 1, 28, 28])
Shape of y: torch.Size([8]) torch.int64


In [20]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [21]:
from LeNet import LeNet5
model = LeNet5()
model.to(device)

LeNet5(
  (net): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): Sigmoid()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Sigmoid()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.437720  [    8/60000]
loss: 2.250335  [  808/60000]
loss: 2.326287  [ 1608/60000]
loss: 2.305676  [ 2408/60000]
loss: 2.329643  [ 3208/60000]
loss: 2.260810  [ 4008/60000]
loss: 2.271054  [ 4808/60000]
loss: 2.280674  [ 5608/60000]
loss: 2.315647  [ 6408/60000]
loss: 2.355812  [ 7208/60000]
loss: 2.307366  [ 8008/60000]
loss: 2.310968  [ 8808/60000]
loss: 2.297341  [ 9608/60000]
loss: 2.281241  [10408/60000]
loss: 2.294526  [11208/60000]
loss: 2.316801  [12008/60000]
loss: 2.305168  [12808/60000]
loss: 2.313088  [13608/60000]
loss: 2.275728  [14408/60000]
loss: 2.296066  [15208/60000]
loss: 2.327045  [16008/60000]
loss: 2.299820  [16808/60000]
loss: 2.292435  [17608/60000]
loss: 2.292693  [18408/60000]
loss: 2.296309  [19208/60000]
loss: 2.292536  [20008/60000]
loss: 2.313410  [20808/60000]
loss: 2.310635  [21608/60000]
loss: 2.309846  [22408/60000]
loss: 2.285259  [23208/60000]
loss: 2.297608  [24008/60000]
loss: 2.308154  [24808/60000]


KeyboardInterrupt: 